In [103]:
# modifying the ast_note_interactivity kernel option to make
# jupyter pretty-print any variable or statement on it’s own line, so you can see the value of multiple statements at once.

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# So that plots are displayed inline, "plt.show()" not needed.
%pylab inline

import sys
print('Python: {}'.format(sys.version))
# scipy
import scipy
print('scipy: {}'.format(scipy.__version__))
# numpy
import numpy
print('numpy: {}'.format(numpy.__version__))
# matplotlib
import matplotlib as plt
print('matplotlib: {}'.format(plt.__version__))
# pandas
import pandas as pd
print('pandas: {}'.format(pd.__version__))
# scikit-learn
import sklearn
print('sklearn: {}'.format(sklearn.__version__))
import re
import seaborn as sns

pd.options.display.max_rows=6
plt.rc('figure', figsize=(10, 6))

Populating the interactive namespace from numpy and matplotlib
Python: 3.5.2 |Anaconda custom (x86_64)| (default, Jul  2 2016, 17:52:12) 
[GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]
scipy: 0.17.1
numpy: 1.11.1
matplotlib: 1.5.1
pandas: 0.19.0
sklearn: 0.18.1


`%matplotlib` prevents importing * from pylab and numpy


# 1. Fuzzywuzzy, Jaccard similarity
# 2. Algorithm 2
#SequenceMatcher, Ratcliff/___
# 3 Jaccard simple pure-python implementation
 From https://medium.com/@sumn2u/advance-sentence-matching-in-python-c78d86f65aa7
 Explained
https://bommaritollc.com/2014/06/30/advanced-approximate-sentence-matching-python/
 
# Algorithm 4
Sentence Similarity Based on Semantic Nets and Corpus Statistics. This implementation is built with Python and Natural Language Tool Kit (NLTK). The Semantic Net referred to in the paper is Wordnet and the Corpus Statistics are from the Brown Corpus, both of which are available using NLTK's corpus API.

From http://sujitpal.blogspot.com/2014/12/semantic-similarity-for-short-sentences.html

In [149]:
from openpyxl import load_workbook
wb = load_workbook(filename = './stringmatch.xlsx')
print('sheet names',wb.get_sheet_names())
sheet1 = wb.get_sheet_by_name('Task')
print('max row and column',sheet1.max_row,sheet1.max_column)
sheet2 = wb.get_sheet_by_name('ReferenceValues')

nrows=19 #one higher bc of header row
columns=['Test_String']
for j in range(2,10):
    columns.append(sheet1.cell(row=1,column=j).value)
#print(len(columns))
#print(columns)

testL=[]
for i in range(2,nrows+2):
    testL.append(sheet1.cell(row=i,column=1).value)
#print('testL len', len(testL))

# reference strings
refL=[]
for i in range(2,sheet2.max_row+1):
    refL.append(sheet2.cell(row=i,column=1).value)
#refL

sheet names ['Task', 'ReferenceValues']
max row and column 1121 9


In [133]:
# Function that runs algorithms
def run_algo(refL,testL,algonum):
    print('running algo',algonum)
    match=[]
    conf=[]
    if algonum==1:
        for test in testL:
            result=process.extractOne(test, refL)
            match.append(result[0])
            conf.append(round(result[1],1))
        return [match,conf]
    if algonum==2:
        print('entered 2')
        for test in testL:
            seq=pd.DataFrame(index=refL, columns=['match','accuracy'])
            for ref in refL:
                matching=difflib.SequenceMatcher(None, test,ref)
                seq.loc[ref,'match']=ref
                seq.loc[ref,'accuracy']=matching.real_quick_ratio()*100.
            max_accu= max(seq['accuracy'])
            max_match= seq.loc[seq['accuracy']==max_accu,'match'][0]
            max_accu=round(max_accu,1)
            conf.append(max_accu)
            match.append(max_match)
        return [match,conf]
    if algonum==2:
        print('entered 2')
        for test in testL:
            seq=pd.DataFrame(index=refL, columns=['match','accuracy'])
            for ref in refL:    
    
    
    
    
    
    print('exiting run_algo')


matchL2,confL2=run_algo(refL,testL,2)

running algo 2
entered 2


In [154]:
print(confL2[1:15])
print('mean confL',np.mean(confL2))

[85.7, 55.5, 100.0, 80.0, 44.7, 100.0, 100.0, 44.7, 100.0, 100.0, 100.0, 100.0, 0.0, 73.0]
mean confL 66.7656565657


In [155]:
#Algorithm 3
#simple pure-Python implementation of Cosine similarity

import re, math
from collections import Counter

WORD = re.compile(r'\w+')

def get_cosine(vec1, vec2):
     intersection = set(vec1.keys()) & set(vec2.keys())
     numerator = sum([vec1[x] * vec2[x] for x in intersection])

     sum1 = sum([vec1[x]**2 for x in vec1.keys()])
     sum2 = sum([vec2[x]**2 for x in vec2.keys()])
     denominator = math.sqrt(sum1) * math.sqrt(sum2)

     if not denominator:
        return 0.0
     else:
        return float(numerator) / denominator

def text_to_vector(text):
     words = WORD.findall(text)
     return Counter(words)


# text1 = 'This is a foo bar sentence .'
# text2 = 'This sentence is similar to a foo bar sentence .'
# vector1 = text_to_vector(text1)
# vector2 = text_to_vector(text2)
# cosine = get_cosine(vector1, vector2)
# print('Cosine:', cosine)

In [160]:
matchL,confL=run_algo(refL,testL,1)
print(confL[1:5])
print('mean confL',np.mean(confL))

running algo 1
[94, 60, 99, 96]
mean confL 89.2105263158


In [161]:
matchL,confL=run_algo(refL,testL,2)
print(confL[1:5])
print('mean confL',np.mean(confL))

running algo 2
entered 2
[99.1, 18.2, 99.2, 100.0]
mean confL 88.8368421053


In [158]:
matchL,confL=run_algo(refL,testL,3)

running algo 3
entered 3
len 19


In [159]:
print(confL[1:15])
print('mean confL',np.mean(confL))

[85.7, 55.5, 100.0, 80.0, 44.7, 100.0, 100.0, 44.7, 100.0, 100.0, 100.0, 100.0, 0.0, 73.0]
mean confL 80.2315789474


# Algorithm 4
Sentence Similarity Based on Semantic Nets and Corpus Statistics. This implementation is built with Python and Natural Language Tool Kit (NLTK). The Semantic Net referred to in the paper is Wordnet and the Corpus Statistics are from the Brown Corpus, both of which are available using NLTK's corpus API.

From http://sujitpal.blogspot.com/2014/12/semantic-similarity-for-short-sentences.html

In [74]:
# Algorithm 4
# 
from __future__ import division
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import brown
import math
import numpy as np
import sys

# Parameters to the algorithm. Currently set to values that was reported
# in the paper to produce "best" results.
ALPHA = 0.2
BETA = 0.45
ETA = 0.4
PHI = 0.2
DELTA = 0.85

brown_freqs = dict()
N = 0

######################### word similarity ##########################

def get_best_synset_pair(word_1, word_2):
    """ 
    Choose the pair with highest path similarity among all pairs. 
    Mimics pattern-seeking behavior of humans.
    """
    #print('inside get_best')
    max_sim = -1.0
    synsets_1 = wn.synsets(word_1)
    synsets_2 = wn.synsets(word_2)
    if len(synsets_1) == 0 or len(synsets_2) == 0:
        return None, None
    else:
        max_sim = -1.0
        best_pair = None, None
        for synset_1 in synsets_1:
            for synset_2 in synsets_2:
                sim = wn.path_similarity(synset_1, synset_2)
                #print('sim',sim, 'max_sim',max_sim, word_1)
                if sim is not None:
                    #print('before sim>max_sim')
                    if sim > max_sim:
                        max_sim = sim
                        best_pair = synset_1, synset_2
        return best_pair
#print("past get best")
def length_dist(synset_1, synset_2):
    """
    Return a measure of the length of the shortest path in the semantic 
    ontology (Wordnet in our case as well as the paper's) between two 
    synsets.
    """
    #print('inside length_dist')
    l_dist = sys.maxsize
    if synset_1 is None or synset_2 is None:
        return 0.0
    if synset_1 == synset_2:
        # if synset_1 and synset_2 are the same synset return 0
        l_dist = 0.0
    else:
        wset_1 = set([str(x.name()) for x in synset_1.lemmas()])        
        wset_2 = set([str(x.name()) for x in synset_2.lemmas()])
        if len(wset_1.intersection(wset_2)) > 0:
            # if synset_1 != synset_2 but there is word overlap, return 1.0
            l_dist = 1.0
        else:
            # just compute the shortest path between the two
            l_dist = synset_1.shortest_path_distance(synset_2)
            if l_dist is None:
                l_dist = 0.0
    # normalize path length to the range [0,1]
    return math.exp(-ALPHA * l_dist)

def hierarchy_dist(synset_1, synset_2):
    """
    Return a measure of depth in the ontology to model the fact that 
    nodes closer to the root are broader and have less semantic similarity
    than nodes further away from the root.
    """
    #print('inside hierarchy')
    h_dist = sys.maxsize
    if synset_1 is None or synset_2 is None:
        return h_dist
    if synset_1 == synset_2:
        # return the depth of one of synset_1 or synset_2
        h_dist = max([x[1] for x in synset_1.hypernym_distances()])
    else:
        # find the max depth of least common subsumer
        hypernyms_1 = {x[0]:x[1] for x in synset_1.hypernym_distances()}
        hypernyms_2 = {x[0]:x[1] for x in synset_2.hypernym_distances()}
        lcs_candidates = set(hypernyms_1.keys()).intersection(
            set(hypernyms_2.keys()))
        if len(lcs_candidates) > 0:
            lcs_dists = []
            for lcs_candidate in lcs_candidates:
                lcs_d1 = 0
                #if hypernyms_1.has_key(lcs_candidate):
                if lcs_candidate in hypernyms_1:
                    lcs_d1 = hypernyms_1[lcs_candidate]
                lcs_d2 = 0
                if lcs_candidate in hypernyms_2:
                    lcs_d2 = hypernyms_2[lcs_candidate]
                lcs_dists.append(max([lcs_d1, lcs_d2]))
            h_dist = max(lcs_dists)
        else:
            h_dist = 0
    return ((math.exp(BETA * h_dist) - math.exp(-BETA * h_dist)) / 
        (math.exp(BETA * h_dist) + math.exp(-BETA * h_dist)))
    
def word_similarity(word_1, word_2):
    synset_pair = get_best_synset_pair(word_1, word_2)
    #print('Inside word_similarity', word_1, word_2)
    return (length_dist(synset_pair[0], synset_pair[1]) * 
            hierarchy_dist(synset_pair[0], synset_pair[1]))

######################### sentence similarity ##########################

def most_similar_word(word, word_set):
    """
    Find the word in the joint word set that is most similar to the word
    passed in. We use the algorithm above to compute word similarity between
    the word and each word in the joint word set, and return the most similar
    word and the actual similarity value.
    """
    #print('inside most_similar')
    max_sim = -1.0
    sim_word = ""
    for ref_word in word_set:
        #print('entering word_similarity')
        sim = word_similarity(word, ref_word)
        if sim is not None:
            if sim > max_sim:
                max_sim = sim
                sim_word = ref_word
    #print('most_similar return sim max',sim_word, max_sim)
    return sim_word, max_sim
    
def info_content(lookup_word):
    """
    Uses the Brown corpus available in NLTK to calculate a Laplace
    smoothed frequency distribution of words, then uses this information
    to compute the information content of the lookup_word.
    """
    #print('inside info_content')
    global N
    if N == 0:
        # poor man's lazy evaluation
        for sent in brown.sents():
            for word in sent:
                word = word.lower()
                if word not in brown_freqs:
                    brown_freqs[word] = 0
                brown_freqs[word] = brown_freqs[word] + 1
                N = N + 1
    lookup_word = lookup_word.lower()
    n = 0 if lookup_word not in brown_freqs else brown_freqs[lookup_word]
    return 1.0 - (math.log(n + 1) / math.log(N + 1))
    
def semantic_vector(words, joint_words, info_content_norm):
    """
    Computes the semantic vector of a sentence. The sentence is passed in as
    a collection of words. The size of the semantic vector is the same as the
    size of the joint word set. The elements are 1 if a word in the sentence
    already exists in the joint word set, or the similarity of the word to the
    most similar word in the joint word set if it doesn't. Both values are 
    further normalized by the word's (and similar word's) information content
    if info_content_norm is True.
    """
    sent_set = set(words)
    semvec = np.zeros(len(joint_words))
    i = 0
    for joint_word in joint_words:
        if joint_word in sent_set:
            # if word in union exists in the sentence, s(i) = 1 (unnormalized)
            semvec[i] = 1.0
            if info_content_norm:
                semvec[i] = semvec[i] * math.pow(info_content(joint_word), 2)
        else:
            # find the most similar word in the joint set and set the sim value
            sim_word, max_sim = most_similar_word(joint_word, sent_set)
            semvec[i] = PHI if max_sim > PHI else 0.0
            if info_content_norm:
                semvec[i] = semvec[i] * info_content(joint_word) * info_content(sim_word)
        i = i + 1
    return semvec                
            
def semantic_similarity(sentence_1, sentence_2, info_content_norm):
    """
    Computes the semantic similarity between two sentences as the cosine
    similarity between the semantic vectors computed for each sentence.
    """
    words_1 = nltk.word_tokenize(sentence_1)
    words_2 = nltk.word_tokenize(sentence_2)
    joint_words = set(words_1).union(set(words_2))
    vec_1 = semantic_vector(words_1, joint_words, info_content_norm)
    vec_2 = semantic_vector(words_2, joint_words, info_content_norm)
    if (np.linalg.norm(vec_1) * np.linalg.norm(vec_2)) > 0:
        return np.dot(vec_1, vec_2.T) / (np.linalg.norm(vec_1) * np.linalg.norm(vec_2))
    else:
        return 0.0
######################### word order similarity ##########################

def word_order_vector(words, joint_words, windex):
    """
    Computes the word order vector for a sentence. The sentence is passed
    in as a collection of words. The size of the word order vector is the
    same as the size of the joint word set. The elements of the word order
    vector are the position mapping (from the windex dictionary) of the 
    word in the joint set if the word exists in the sentence. If the word
    does not exist in the sentence, then the value of the element is the 
    position of the most similar word in the sentence as long as the similarity
    is above the threshold ETA.
    """
    wovec = np.zeros(len(joint_words))
    i = 0
    wordset = set(words)
    for joint_word in joint_words:
        if joint_word in wordset:
            # word in joint_words found in sentence, just populate the index
            wovec[i] = windex[joint_word]
        else:
            # word not in joint_words, find most similar word and populate
            # word_vector with the thresholded similarity
            sim_word, max_sim = most_similar_word(joint_word, wordset)
            if max_sim > ETA:
                wovec[i] = windex[sim_word]
            else:
                wovec[i] = 0
        i = i + 1
    return wovec

def word_order_similarity(sentence_1, sentence_2):
    """
    Computes the word-order similarity between two sentences as the normalized
    difference of word order between the two sentences.
    """
    #print('inside word_order_similarity')
    words_1 = nltk.word_tokenize(sentence_1)
    words_2 = nltk.word_tokenize(sentence_2)
    joint_words = list(set(words_1).union(set(words_2)))
    windex = {x[1]: x[0] for x in enumerate(joint_words)}
    r1 = word_order_vector(words_1, joint_words, windex)
    r2 = word_order_vector(words_2, joint_words, windex)
    if np.linalg.norm(r1 + r2) > 0:
        return 1.0 - (np.linalg.norm(r1 - r2) / np.linalg.norm(r1 + r2))
    else:
        return 0.0

######################### overall similarity ##########################

def similarity(sentence_1, sentence_2, info_content_norm):
    """
    Calculate the semantic similarity between two sentences. The last 
    parameter is True or False depending on whether information content
    normalization is desired or not.
    """
    #print('inside similarity',  (1.0 - DELTA) * word_order_similarity(sentence_1, sentence_2))
    #print('Delta',DELTA)
    similarity_score= DELTA * semantic_similarity(sentence_1, sentence_2, info_content_norm) + \
        (1.0 - DELTA) * word_order_similarity(sentence_1, sentence_2)
    if math.isnan(similarity_score): 
        return 0.0
    else:
        return similarity_score

In [217]:
mystr="bla   "
len(mystr.replace(" ",""))>=2

True

In [233]:
# Function that runs algorithms
def run_algo(refL,testL,algonum):
    #print('running algo',algonum)
    match=[]
    conf=[]
    if algonum==1:
        for test in testL:
            test1=str(test).replace(" ","")
            if len(test1)>=7:
                result=process.extractOne(test, refL)
                match.append(result[0])
                conf.append(round(result[1],1))
            else:
                match.append('No Match')
                conf.append(0.0)
        return [match,conf]
    if algonum==2:
        #print('entered 2')
        for test in testL:
            test1=str(test).replace(" ","")
            if len(test1)>=7:
                seq=pd.DataFrame(index=refL, columns=['match','accuracy'])
                for ref in refL:
                    matching=difflib.SequenceMatcher(None, test,ref)
                    seq.loc[ref,'match']=ref
                    seq.loc[ref,'accuracy']=matching.real_quick_ratio()*100.
                max_accu= max(seq['accuracy'])
                max_match= seq.loc[seq['accuracy']==max_accu,'match'][0]
                max_accu=round(max_accu,1)
                conf.append(max_accu)
                match.append(max_match)
            else:
                match.append('No Match')
                conf.append(0.0)
        return [match,conf]
    if algonum==3:
        #print('entered 3')
        for test in testL:
            seq=pd.DataFrame(index=refL, columns=['match','accuracy'])
            test1=str(test).replace(" ","")
            if len(test1)>=7:
                for ref in refL:    
                    testVec=text_to_vector(test)
                    refVec =text_to_vector(ref)
                    seq.loc[ref,'match']=ref
                    seq.loc[ref,'accuracy']=get_cosine(testVec, refVec)*100 
                    #print('accuracy',seq.loc[ref,'accuracy'])
                max_accu= max(seq['accuracy'])
                max_match= seq.loc[seq['accuracy']==max_accu,'match'][0]
                max_accu=round(max_accu,1)
                #print('accu',max_accu)
                conf.append(max_accu)
                match.append(max_match)
            else:
                match.append('No Match')
                conf.append(0.0)
        #print('len',len(conf))
        print('exiting algo 3')
        return [match,conf]
    if algonum==4:
        print('entered 4')
        counter=0
        for test in testL:
            seq=pd.DataFrame(index=refL, columns=['match','accuracy'])
            test1=str(test).replace(" ","")
            if len(test1)>=7:
                for ref in refL:    
                    testVec=text_to_vector(test)
                    refVec =text_to_vector(ref)
                    seq.loc[ref,'match']=ref
                    seq.loc[ref,'accuracy']=similarity(test, ref, False)*100
                    #set to True to have words normalized for Information Content. 
                    #print('accuracy',seq.loc[ref,'accuracy'])
                max_accu= max(seq['accuracy'])
                max_match= seq.loc[seq['accuracy']==max_accu,'match'][0]
                max_accu=round(max_accu,1)
                conf.append(max_accu)
                match.append(max_match)
            else:
                match.append('No Match')
                conf.append(0.0)
            counter=counter+1
            print('counter',counter)
        print('len 4',len(conf))
        return [match,conf]       
    print('exiting run_algo')

In [234]:
allrows=1121
#Batches of 100
#nrows=99
nrows=300 #one higher bc of header row
print('nrows',nrows)
ilist=list(range(100,allrows,100))
ilist.append(allrows)

#read data
wb = load_workbook(filename = './stringmatch.xlsx')
print('sheet names',wb.get_sheet_names())
sheet1 = wb.get_sheet_by_name('Task')
print('max row and column',sheet1.max_row,sheet1.max_column)
sheet2 = wb.get_sheet_by_name('ReferenceValues')
# full list of reference strings
refL=[]
for i in range(2,sheet2.max_row+1):
    refL.append(sheet2.cell(row=i,column=1).value)
#refL
# To make the task dataframe
columns=['Test_String']
for j in range(2,10):
    columns.append(sheet1.cell(row=1,column=j).value)
#print(len(columns))
#print(columns)
#Pandas data frame starts with index 0 = cell A1
task=pd.DataFrame(index=range(0,nrows+1), columns=columns)

nrows 300
sheet names ['Task', 'ReferenceValues']
max row and column 1121 9


In [235]:
#partial list of test strings
testL=[]
#for i in ilist:
for i in range(2,nrows+2):
    testL.append(sheet1.cell(row=i,column=1).value)
#print('testL len', len(testL))
task.iloc[0:nrows,0]=testL

In [236]:
# RUN ALGORITHMS 1 AND 2
# run 4 algos
matchL,confL=run_algo(refL,testL,1)
print('len confL',len(confL))
#print('len confL',len(task.iloc[1:21,1]))
task.iloc[0:nrows,1]=matchL
task.iloc[0:nrows,2]=confL

matchL,confL=run_algo(refL,testL,2)
print('algo 2 len confL',len(confL))
#print('len confL',len(task.iloc[1:21,1]))
task.iloc[0:nrows,3]=matchL
task.iloc[0:nrows,4]=confL

len confL 300
len confL 300


In [237]:
# ALGORITHMS 3 AND 4

matchL,confL=run_algo(refL,testL,3)
print('algo 3 len confL',len(confL))
#print('len confL',len(task.iloc[1:21,1]))
task.iloc[0:nrows,5]=matchL
task.iloc[0:nrows,6]=confL

matchL,confL=run_algo(refL,testL,4)
print('algo 4 len confL',len(confL))
#print('len confL',len(task.iloc[1:21,1]))
task.iloc[0:nrows,7]=matchL
task.iloc[0:nrows,8]=confL


#write data to table
print('testL',task.iloc[0:3,0])
#print('matchL',task.iloc[0:3,1])
print('last confL'+'\n',task.iloc[nrows-3:nrows,8])

    #write table to file

    #write table to excel

exiting algo 3
algo 3 len confL 300
entered 4
len 4 300
algo 4 len confL 300
testL 0    RESULT NEGATIVE  NO CLINICALLY SIGNIFICANT MUT...
1    RESULT NEGATIVE  NO CLINICALLY SIGNIFICANT MUt...
2                                                    A
Name: Test_String, dtype: object
last confL 297    100
298    100
299      0
Name: Algo4Confidence, dtype: object


In [301]:
mean_1=np.mean(task.iloc[:,2])
mean_2=np.mean(task.iloc[:,4])
mean_3=np.mean(task.iloc[:,6])
mean_4=np.mean(task.iloc[:,8])
print(mean_4)

means=['average accuracy',"",mean_1,"",mean_2,"",mean_3,"",mean_4]

67.0993333333


In [302]:
#replace task2 with task
import re
import xlwt
import xlsxwriter

shape(task)

task2=task.iloc[0:300,0:11]

# Create a workbook and add a worksheet.
workbook = xlsxwriter.Workbook('matched.xlsx')
worksheet = workbook.add_worksheet("Task")
# Add formats
bold = workbook.add_format({'bold': True})
percent = workbook.add_format({'num_format': '0%'})
# Header
for i in range(1,10):
    print('before write')
    worksheet.write(0, i-1, columns[i-1], bold)
    print('columns[i-1]',columns[i-1], i-1)
for row in task2.iterrows():
    rn=int(row[0])+1
    cn=1
    print('rn',rn,'shape',shape(task2))
    for col in task2:
        #worksheet.write(rn, cn, task2.iloc[rn,cn])
        worksheet.write(rn, cn-1, task2.iloc[rn-1,cn-1])
        #print('\n')
        #print('cn',cn)
        #print(task2[col][rn])
        cn=cn+1
        #print('cn',cn)
        if cn>= shape(task2)[1]:
            print('before 1st return')
            break
    if rn>= shape(task2)[0]-1:
        print('rn max')
        break
for i in range(1,10):
    worksheet.write(305, i-1, means[i-1], bold)
workbook.close()



(301, 9)

before write


0

columns[i-1] Test_String 0
before write


0

...

0

0

0

0

0

In [268]:
shape(task)

task2=task.iloc[0:3,0:11]

(301, 9)

In [259]:
print(task.iloc[0,0])
print(task.items())

RESULT NEGATIVE  NO CLINICALLY SIGNIFICANT MUTATION IDENTIFIED
<generator object DataFrame.iteritems at 0x11e41c990>


In [254]:
for row in task2.iterrows():
    rn=int(row[0])
    cn=0
    print('rn',rn)
    for col in task2:
        worksheet.write(rn, cn, task2.iloc[rn,cn])
        print('\n')
        print('cn',cn)
        #print(task2[col][rn])
        cn=cn+1

rn 0


cn 0


cn 1


cn 2


cn 3


cn 4


cn 5
rn 1


cn 0


cn 1


cn 2


cn 3


cn 4


cn 5
rn 2


cn 0


cn 1


cn 2


cn 3


cn 4


cn 5


In [260]:
print(columns)

['Test_String', 'Algo1Match', 'Algo1Confidence', 'Algo2Match', 'Algo2Confidence', 'Algo3Match', 'Algo3Confidence', 'Algo4Match', 'Algo4Confidence']


In [262]:
for i in range(1,10):
    print(columns[i-1])

Test_String
Algo1Match
Algo1Confidence
Algo2Match
Algo2Confidence
Algo3Match
Algo3Confidence
Algo4Match
Algo4Confidence
